# Simulation of Schwinger model

In this notebook, we investigate realizing (1 + 1)-dimensional quantum electrodynamics (the Schwinger model) on a few-qubit trapped-ion quantum computer. We implement and deploy the algorithm developed in [[1605.04570]](https://arxiv.org/abs/1605.04570). 

## 1. Introduction


## 2. Hamiltonian to be simulated

$$J\sum_{i<j} c_{ij} Z_iZ_j+ \frac{w}{2}\sum_i (X_iX_{i+1}+Y_iY_{i+1})+m\sum_i c_iZ_i+J\sum_i \tilde{c}_iZ_i $$

The challenge of this model is that the Hamiltonian is not a sum of local terms, it requires long range interactions. Fortunately, the trapped-ion quantum computer can be programmed to implement the long range interactions.



In [1]:
# Generate a Schwinger model in pure spin Hamiltonian

import numpy as np
from simuq.qsystem import QSystem
from simuq.environment import qubit

def Schwinger(n, T = 1, m = 0.5, w = 1, J = 1, eps_0 = 0) :
    qs = QSystem()
    q = [qubit(qs) for i in range(n)]
    H = 0
    for j in range(n) :
        H += m / 2 * (-1)**j * q[j].Z
    for j in range(n - 1) :
        H += w / 2 * (q[j].X * q[j+1].X + q[j].Y * q[j+1].Y)
    for j in range(n - 1) :
        S = 0
        for k in range(j) :
            S += q[k].Z + (-1)**k
        H = J * (eps_0 + S) * (eps_0 + S)
    qs.add_evolution(H, T)
    return qs

N=4

The initial state is $| 1010\rangle$

In [1]:
from simuq.providers import IonQProvider

ionqp = IonQProvider(from_file='../../../ionq_API_key')

In [3]:
def Xgate(q) :
    gate = {"gate" : "gpi", "target" : q, "phase" : 0.}
    return gate
state_prep = {'circuit': [Xgate(0), Xgate(2)]}

In [ ]:
import matplotlib.pyplot as plt
def draw_results(steps, results_list) :
    density_list=[]
    for result in results_list:
        density=0
        for state in result:
            density+=result[state]*sum(1 for a, b in zip(state, "1010") if a != b)
        density_list.append(density)
    plt.plot(steps, density_list, 'o-')

In [ ]:
import time
timestep=np.linspace(0, np.pi/2, 20)
results_list = []
for t in timestep :
    walk = Schwinger(N, t)
    ionqp.compile(walk, backend='aria-1', trotter_num = 14, state_prep = state_prep)
    ionqp.run(on_simulator = True, verbose = -1)
    while True :
        time.sleep(5)
        res = ionqp.results(verbose = -1)
        if res != None :
            break
    results_list.append(res)

draw_results(timestep, results_list, True)

In [ ]:
from simuq.providers import QuTiPProvider

qtpp = QuTiPProvider()
qtpp.compile(walk)